# Sagemaker Blue/Green Deployment

1. [Introduction](#Introduction)
2. [Prerequisites and Preprocessing](#Prequisites-and-Preprocessing)
3. [Training parameters for HPO](#Training-parameters-for-HPO)
4. [Plot training and validation accuracies](#Plot-training-and-validation-accuracies)


## Introduction
***

In this notebook, we use the Blue/Green deployment feature integrated in Sagemaker in order to deploy new models in production without impacting the end user's user experience.

## Prequisites and Preprocessing
***
### Permissions and environment variables

Before starting this notebook, it is mandatory to do the following notebooks : 

- 1 - Word2vec-HPO

## Setup Green Environment
***
** You can request the endpoint in a loop with inference.py **
### How Blue/Green Deployment works on Sagemaker?

![Sagemaker BlueGreen](img/SagemakerBlueGreen.png)

#### Pick the best model artifact returned from the 2nd notebooks (HPO) and create a Sagemaker Model

In [ ]:
user='user1'
my_bucket='marc-stationf-sagemaker'
endpoint_name='blazingtext-2018-10-08-14-55-37-874'

In [ ]:
%%time
import boto3
import re
import time
from time import gmtime, strftime
from sagemaker import get_execution_role

role = get_execution_role()

bucket = my_bucket # customize to your bucket

containers = {'us-west-2': '433757028032.dkr.ecr.us-west-2.amazonaws.com/blazingtext:latest',
              'us-east-1': '811284229777.dkr.ecr.us-east-1.amazonaws.com/blazingtext:latest',
              'us-east-2': '825641698319.dkr.ecr.us-east-2.amazonaws.com/blazingtext:latest',
              'eu-west-1': '685385470294.dkr.ecr.eu-west-1.amazonaws.com/blazingtext:latest'}
training_image = containers[boto3.Session().region_name]
print(training_image)


In [ ]:
# Create model B
job_name_prefix_modelb = 'blazin-modelB-' + user
timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
job_name_modelb = job_name_prefix_modelb + timestamp
s3_output_modelb = 's3://marc-stationf-sagemaker/blazin-training-user1/output/blazin-hpo-user1-22-34-58-002-64ece8c9/output/model.tar.gz' ### Copy the result from HPO notebooks

sagemaker = boto3.client(service_name='sagemaker')

hosting_image = containers[boto3.Session().region_name]
primary_container = {
    'Image': hosting_image,
    'ModelDataUrl': s3_output_modelb,
}

create_model_response = sagemaker.create_model(
    ModelName = job_name_modelb,
    ExecutionRoleArn = role,
    PrimaryContainer = primary_container)

print(create_model_response['ModelArn'])

#### Create a new endpoint configuration for Blue/Green environment

In [ ]:
from datetime import datetime
job_name_modela_prefix = 'blazingtext-2018-10-08-22-19-22-522'
response = sagemaker.list_models(
    NameContains=job_name_modela_prefix,
    CreationTimeAfter=datetime(2018, 9, 12)
)
job_name_modela = response['Models'][0]['ModelName']

In [ ]:
job_name_prefix = 'blazin-modelAB-' + user

timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
endpoint_config_name = job_name_prefix + '-bluegreen' + timestamp

endpoint_config_response = sagemaker.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[
        {
        'InstanceType':'ml.m4.xlarge',
        'InitialInstanceCount':1,
        'ModelName':job_name_modela,
        'VariantName':'Model-A',
        'InitialVariantWeight':1
        },
        {
        'InstanceType':'ml.m4.xlarge',
        'InitialInstanceCount':1,
        'ModelName':job_name_modelb,
        'VariantName':'Model-B',
        'InitialVariantWeight':0
        }
    ])

print('Endpoint configuration name: {}'.format(endpoint_config_name))
print('Endpoint configuration arn:  {}'.format(endpoint_config_response['EndpointConfigArn']))

#### Update endpoint with new endpoint configuration

In [ ]:
endpoint_config_response = sagemaker.update_endpoint(
    EndpointName = endpoint_name,
    EndpointConfigName = endpoint_config_name)

print(endpoint_config_response)

#### Update endpoint weight and capacities

In [ ]:
response = sagemaker.update_endpoint_weights_and_capacities(
    EndpointName=endpoint_name,
    DesiredWeightsAndCapacities=[
        {
            'VariantName': 'Model-A',
            'DesiredWeight': 0,
        },
        {
            'VariantName': 'Model-B',
            'DesiredWeight': 1,
        }
    ]
)

#### Create green endpoint configuration

In [ ]:
job_name_prefix = 'blazin-modelB-' + user

timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
endpoint_config_name = job_name_prefix + '-green' + timestamp

endpoint_config_response = sagemaker.create_endpoint_config(
    EndpointConfigName = endpoint_config_name,
    ProductionVariants=[
        {
        'InstanceType':'ml.m4.xlarge',
        'InitialInstanceCount':1,
        'ModelName':job_name_modelb,
        'VariantName':'Model-B',
        'InitialVariantWeight':1
        }
    ])

print('Endpoint configuration name: {}'.format(endpoint_config_name))
print('Endpoint configuration arn:  {}'.format(endpoint_config_response['EndpointConfigArn']))

#### Update endpoint with green endpoint configuration

In [ ]:
endpoint_config_response = sagemaker.update_endpoint(
    EndpointName = endpoint_name,
    EndpointConfigName = endpoint_config_name)

print(endpoint_config_response)